In [ ]:
# Import necessary librariesimport osfrom pathlib import Pathfrom PyPDF2 import PdfReaderfrom langchain_text_splitters import RecursiveCharacterTextSplitterfrom langchain_openai import OpenAIEmbeddingsfrom langchain_community.vectorstores import FAISSfrom langchain_openai import ChatOpenAIfrom langchain_core.prompts import PromptTemplateimport config

In [ ]:
# Set OpenAI API keyos.environ["OPENAI_API_KEY"] = config.API_KEY

In [ ]:
# PDF reading functiondef read_pdf(pdf_path):    reader = PdfReader(str(pdf_path))    pages_text = [page.extract_text() or "" for page in reader.pages]    full_text = "\n\n".join(pages_text)    return full_text

In [ ]:
# Load and process PDFuser_path = Path(r"data/books/sample.pdf")full_text = read_pdf(user_path)# Split text into chunkstext_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=200)chunks = text_splitter.split_text(full_text)

In [ ]:
# Create embeddings and store in a vector databaseembeddings = OpenAIEmbeddings(model="text-embedding-3-small")vector_store = FAISS.from_texts(chunks, embeddings)vector_store.save_local("faiss_vector_store")

In [ ]:
# Define the retrieval functiondef retrieve_answer(question):    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":3})    retrieved_docs = retriever.invoke(question)    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)    return context_text

In [ ]:
# Define the prompt templateprompt = PromptTemplate(    template="""    You are a helpful assistant.    Answer ONLY from the provided context.    If the context is insufficient, just say you don't know.        {context}    Question: {question}    """,    input_variables=["context", "question"])

In [ ]:
# Define the function to get the final answerdef get_final_answer(question):    context = retrieve_answer(question)    final_prompt = prompt.invoke({'context': context, 'question': question})    return final_prompt

# Streamlit ApplicationThis notebook is designed to be used with a Streamlit application that allows users to ask questions based on the content of a PDF.